In [111]:
import bs4 as bs  
import urllib.request  
import re
import nltk
import logging
import re
import nltk
import heapq 

In [21]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.summarization import summarize as gsm
from gensim.summarization import keywords as gkw

url_topull = input('Enter the Wikipedia URL to pull - ')

scraped_data = urllib.request.urlopen(url_topull)  
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:  
    article_text += p.text

Enter the Wikipedia URL to pull - https://en.wikipedia.org/wiki/A_Star_Is_Born_(2018_film)


In [3]:
print(article_text)


A Star Is Born is a 2018 American musical romantic drama film produced and directed by Bradley Cooper (in his directorial debut) and written by Eric Roth, Cooper and Will Fetters. A remake of the 1937 film of the same name, it stars Cooper, Lady Gaga, Dave Chappelle, Andrew Dice Clay, and Sam Elliott, and follows a hard-drinking musician (Cooper) who discovers and falls in love with a young singer (Gaga). It marks the third remake of the original 1937 film, after the 1954 musical and the 1976 musical.
Talks of a remake of A Star Is Born began in 2011, with Clint Eastwood attached to direct and Beyoncé set to star. The film was in development hell for several years with various actors approached to co-star, including Christian Bale, Leonardo DiCaprio, Will Smith, and Tom Cruise. In March 2016, Cooper signed on to star and direct, and Lady Gaga joined the cast in August 2016. Principal photography began at the Coachella Valley Music and Arts Festival in April 2017.
A Star Is Born premie

In [22]:
print (gsm(article_text,ratio=0.05))

2019-03-06 17:14:12,763 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 17:14:12,775 : INFO : built Dictionary(679 unique tokens: ['american', 'born', 'bradlei', 'cooper', 'debut']...) from 82 documents (total 1427 corpus positions)


A Star Is Born premiered at the 75th Venice International Film Festival on August 31, 2018, and was theatrically released in the United States on October 5, 2018, by Warner Bros.
The film received numerous accolades, including eight nominations at the 91st Academy Awards, among them Best Picture, Best Actor (Cooper), Best Actress (Gaga) and Best Supporting Actor (Elliott), and won for Best Original Song ("Shallow").
In January 2011, it was announced that Clint Eastwood was in talks to direct Beyoncé in a third American remake of the 1937 film A Star Is Born; however, the project was delayed due to Beyoncé's pregnancy.[9] In April 2012, writer Will Fetters told Collider that the script was inspired by Kurt Cobain.[10] Talks with Christian Bale, Leonardo DiCaprio, Tom Cruise, Johnny Depp, and Will Smith to play the male lead failed to come to fruition.
Furthermore, the 2013 Bollywood romance film, Aashiqui 2, was inspired by "A Star is Born", but was not a remake.[17] On November 9, 2016

In [23]:
print (gkw(article_text,ratio=0.05))

jack
film
films
filming
ally
cooper
musical
music
best
performances
performance
performs
perform
performed
night
including
included
include
nelson
audiences
song
songs
record
records
recording
lead
leads
gaga
million
rez
ray
weekend
weekends
released
releases
release
bobby
gave
awards
award
time
times
singer
addiction
addict
imax


In [14]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
article_text = re.sub(r'\s+', ' ', article_text)  

formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

sentence_list = nltk.sent_tokenize(article_text)  

stopwords = nltk.corpus.stopwords.words('english')

print("Text pre-processing pull done")
word_frequencies = {}  
for word in nltk.word_tokenize(formatted_article_text):  
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():  
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

print("Word Frequencies determined")

sentence_scores = {}  
for sent in sentence_list:  
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

summary_sentences = heapq.nlargest(10, sentence_scores, key=sentence_scores.get)
print(''.join(summary_sentences))

Text pre-processing pull done
Word Frequencies determined
It also screened at the Toronto International Film Festival, the San Sebastián International Film Festival, and the Zurich Film Festival in September 2018.The film remained in second place in its second, third and fourth weekends, grossing a respective $28 million, $19.3 million and $14.1 million.It was chosen by both the National Board of Review and American Film Institute as one of the Top 10 Films of 2018.The film received five nominations at the 76th Golden Globe Awards, including Best Motion Picture – Drama, and 8 nominations at the 91st Academy Awards, including Best Picture.It marks the third remake of the original 1937 film, after the 1954 musical and 1976 rock musical.It marks the third remake of the original 1937 film, after the 1954 musical and the 1976 musical.Audiences polled by CinemaScore gave the film an average grade of "A" on an A+ to F scale, while PostTrak reported film-goers gave it a 90% positive score.Afte

In [19]:
#Measuring using ROGUE
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(''.join(summary_sentences),article_text)
scores

[{'rouge-1': {'f': 0.228702991067358, 'p': 1.0, 'r': 0.1291161178509532},
  'rouge-2': {'f': 0.16262135768205194,
   'p': 0.9663461538461539,
   'r': 0.0887809187279152},
  'rouge-l': {'f': 0.130986661389904, 'p': 1.0, 'r': 0.1291161178509532}}]

In [17]:
scores = rouge.get_scores(article_text,summarize(article_text,ratio=0.05))
scores

2019-03-06 15:09:44,808 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 15:09:44,813 : INFO : built Dictionary(679 unique tokens: ['american', 'born', 'bradlei', 'cooper', 'debut']...) from 109 documents (total 1427 corpus positions)


[{'rouge-1': {'f': 0.1883830438195131, 'p': 0.10398613518197573, 'r': 1.0},
  'rouge-2': {'f': 0.13113401937887897,
   'p': 0.07022968197879859,
   'r': 0.9875776397515528},
  'rouge-l': {'f': 0.10499249430742699, 'p': 0.10398613518197573, 'r': 1.0}}]

In [30]:
from summa import summarizer as ssm
print(ssm.summarize(article_text,ratio= 0.05))

announced that Cooper was in final talks to make his directorial debut with the film, and potentially to star with Beyoncé, who was again in talks to join;[12] Cooper did become the male lead.[13][14][15] On August 16, 2016, it was reported that Lady Gaga had officially become attached and the studio had green-lit the project to begin production early 2017.[16] It marks the third remake of the original 1937 film, after the 1954 musical and 1976 rock musical.
Furthermore, the 2013 Bollywood romance film, Aashiqui 2, was inspired by "A Star is Born", but was not a remake.[17] On November 9, 2016, it was reported that Ray Liotta was in talks to join the film in the role of the manager to Cooper's character, though he ultimately was not involved.[18] On March 17, 2017, Sam Elliott joined the film,[19] with Andrew Dice Clay entering negotiations to play Lorenzo, the father of Lady Gaga's character.[20] Clay was selected over Robert De Niro, John Turturro and John Travolta.[21] In April 2017

## Evaluating based on ROUGE-1 scores

### Based on opinions dataset

In [48]:
import os
direc = "C:/Users/abhin/Documents/3_Spring Classes/Cognitive Computing/Project/Easy techniques/data/summaries-gold"
ext = '.gold'
keys = [i for i in os.listdir(direc)]
gold_dict = {}

for k in keys:
    for i in os.listdir(direc+r'/'+k):
        if os.path.splitext(i)[1] == ext:
            with open(os.path.join(direc + r'/'+ k + r'/'+ i)) as file_object:
                if k not in gold_dict.keys():
                    gold_dict[k] = [file_object.read()]
                else:
                    gold_dict[k].append(file_object.read())

In [107]:
import json
with open('gold_dict.json', 'w') as fp:
    json.dump(gold_dict, fp)

In [32]:
# Iterate over your txt files
direc = "C:/Users/abhin/Documents/3_Spring Classes/Cognitive Computing/Project/Easy techniques/data/topics"
topics_dict = {}
ext = '.data'

txt_files = [i for i in os.listdir(direc) if os.path.splitext(i)[1] == ext]

for f in txt_files:
    # Open them and assign them to file_dict
    with open(os.path.join(direc,f)) as file_object:
        topics_dict[f] = file_object.read()

In [78]:
from gensim.summarization import summarize
summary_dict = {}

for k,v in topics_dict.items():
    temp_key = str.split(k,'.')[0]
    summary_dict[temp_key] = summarize(v,word_count = 100)

In [80]:
# ROGUE scores calculation
from rouge import Rouge
rouge = Rouge()

In [81]:
def create_rogue_scores(summary_dict):
    rogue_dict = {}
    for key,value in gold_dict.items():
        for v in value:
            if key not in rogue_dict.keys():
                rogue_dict[key] = [rouge.get_scores(summary_dict[key],v)]
            else:
                rogue_dict[key].append(rouge.get_scores(summary_dict[key],v))
    
    return(rogue_dict)

In [82]:
gensim_summarize_rogue = create_rogue_scores(summary_dict)

In [98]:
gensim_rouge1_f = []
gensim_rouge1_p = []
gensim_rouge1_r = []
for v in gensim_summarize_rogue.values():
    for i in v:
        for j in i:
            gensim_rouge1_p.append(j['rouge-1']['p'])
            gensim_rouge1_f.append(j['rouge-1']['f'])
            gensim_rouge1_r.append(j['rouge-1']['r'])

In [105]:
max(gensim_rouge1_p)

0.25

In [106]:
sum(gensim_rouge1_p)/len(gensim_rouge1_p)

0.09594143514797672

In [84]:
from summa import summarizer
summary_dict = {}

for k,v in topics_dict.items():
    temp_key = str.split(k,'.')[0]
    summary_dict[temp_key] = summarizer.summarize(v,words = 100)

In [85]:
summa_summarize_rogue = create_rogue_scores(summary_dict)

In [99]:
summa_rouge1_f = []
summa_rouge1_p = []
summa_rouge1_r = []
for v in summa_summarize_rogue.values():
    for i in v:
        for j in i:
            summa_rouge1_p.append(j['rouge-1']['p'])
            summa_rouge1_f.append(j['rouge-1']['f'])
            summa_rouge1_r.append(j['rouge-1']['r'])

In [104]:
max(gensim_rouge1_p)

0.25

In [101]:
sum(summa_rouge1_p)/len(summa_rouge1_p)

0.1260251689296316

Summa perfoms better than gensim

In [113]:
def easy_nltk(article_text,sentence_count = 5):
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
    article_text = re.sub(r'\s+', ' ', article_text)  

    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

    sentence_list = nltk.sent_tokenize(article_text)  

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}  
    for word in nltk.word_tokenize(formatted_article_text):  
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    summary_sentences = heapq.nlargest(10, sentence_scores, key=sentence_scores.get)
    return(''.join(summary_sentences))

In [114]:
summary_dict = {}

for k,v in topics_dict.items():
    temp_key = str.split(k,'.')[0]
    summary_dict[temp_key] = easy_nltk(v,sentence_count = 5)

In [115]:
nltk_summarize_rogue = create_rogue_scores(summary_dict)

In [116]:
nltk_rouge1_f = []
nltk_rouge1_p = []
nltk_rouge1_r = []
for v in summa_summarize_rogue.values():
    for i in v:
        for j in i:
            nltk_rouge1_p.append(j['rouge-1']['p'])
            nltk_rouge1_f.append(j['rouge-1']['f'])
            nltk_rouge1_r.append(j['rouge-1']['r'])

In [119]:
sum(nltk_rouge1_p)/len(nltk_rouge1_p)

0.1260251689296316

In [118]:
max(nltk_rouge1_p)

0.35714285714285715